Demo ipynb for LDA

Testing the pipeline for a single game

In [1]:
import pandas as pd
import numpy as np

from pathlib import Path

import gensim

import spacy

import nltk

import pyLDAvis
import pickle
from datetime import datetime

In [3]:
# download nltk stopwords
# import nltk
# nltk.download('stopwords')

In [2]:
# load a dataset

dataset_path = Path('../../dataset/topic_modelling/top_11_genres/02_Adventure.pkl')

dataset = pd.read_pickle(dataset_path)

dataset.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 636492 entries, 20727 to 4891927
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   index         636492 non-null  int64 
 1   app_id        636492 non-null  int64 
 2   app_name      636492 non-null  object
 3   review_text   636492 non-null  object
 4   review_score  636492 non-null  int64 
 5   review_votes  636492 non-null  int64 
 6   genre_id      636492 non-null  object
 7   category_id   636492 non-null  object
dtypes: int64(4), object(4)
memory usage: 43.7+ MB


In [3]:
# data preprocessing

import re

def clean(raw):
    """ Remove hyperlinks and markup """
    result = re.sub("<[a][^>]*>(.+?)</[a]>", 'Link.', raw)
    result = re.sub('&gt;', "", result)
    result = re.sub('&#x27;', "'", result)
    result = re.sub('&quot;', '"', result)
    result = re.sub('&#x2F;', ' ', result)
    result = re.sub('<p>', ' ', result)
    result = re.sub('</i>', '', result)
    result = re.sub('&#62;', '', result)
    result = re.sub('<i>', ' ', result)
    result = re.sub("\n", '', result)
    return result

def deEmojify(x):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'', x)
    
def remove_num(texts):
   output = re.sub(r'\d+', '', texts)
   return output

def unify_whitespaces(x):
    cleaned_string = re.sub(' +', ' ', x)
    return cleaned_string

from nltk.corpus import stopwords
stop=set(stopwords.words("english"))
def remove_stopword(text):
   text=[word.lower() for word in text.split() if word.lower() not in stop]
   return " ".join(text)

# only keep alphabets
def remove_non_alphabets(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    return text

def cleaning(df, review):
    df[review] = df[review].apply(lambda x: clean(x))
    df[review] = df[review].apply(lambda x: deEmojify(x))
    df[review] = df[review].apply(lambda x: remove_num(x))
    df[review] = df[review].apply(lambda x: unify_whitespaces(x))
    df[review] = df[review].apply(lambda x: remove_non_alphabets(x))
    df[review] = df[review].apply(lambda x: remove_stopword(x))
    df[review] = df[review].apply(lambda x: x.lower())

def cleaning_strlist(str_list):
    str_list = list(map(lambda x: clean(x), str_list))
    str_list = list(map(lambda x: deEmojify(x), str_list))
    str_list = list(map(lambda x: remove_num(x), str_list))
    str_list = list(map(lambda x: unify_whitespaces(x), str_list))
    str_list = list(map(lambda x: remove_non_alphabets(x), str_list))
    str_list = list(map(lambda x: remove_stopword(x), str_list))
    str_list = list(map(lambda x: x.lower(), str_list))
    return str_list

In [4]:
# apply data preprocessing

cleaning(dataset, 'review_text')

In [5]:
X = dataset['review_text'].values

In [6]:
X

array(['get kq collection working windows download patch http www sierrahelp com files patches kingsquest kqcollectiondbupdate exe set install c program files steam steamapps common king quest collection wherever steam saved kq collection files program patch steam version kq collection run dos box confirm makes game joy run win bit cons run game dosbox shortcuts patch creates steam launcher work game mouse cursor become black white otherwise colored',
       'solved http steamcommunity com app discussions years wanting play running one issue another trying get load finally able find solution windows x even windows ran updates able launch games old bit windows glory using scummvm sierra patch already able detect set proper game good sierra full cd quality video kq kq way play years ago pick long games great great review less game google nod sierrahelp patch steam install need limit colors entire system need tweak open new folders made program files x steam steamapps common king quest co

In [7]:
# nltk.download('averaged_perceptron_tagger')
# t = nltk.word_tokenize(X[0])
# tt = nltk.pos_tag(t)
# tt

In [8]:
# do lemmatization, but not stemming (as part of speech is important in topic modelling)
# use nltk wordnet for lemmatization

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemma = WordNetLemmatizer()

# from https://stackoverflow.com/questions/25534214/nltk-wordnet-lemmatizer-shouldnt-it-lemmatize-all-inflections-of-a-word

# from: https://www.cnblogs.com/jclian91/p/9898511.html
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None     # if none -> created as noun by wordnet
    
def lemmatization(text):
   # use nltk to get PoS tag
    tagged = nltk.pos_tag(nltk.word_tokenize(text))

    # then we only need adj, adv, verb, noun
    # convert from nltk Penn Treebank tag to wordnet tag
    wn_tagged = list(map(lambda x: (x[0], get_wordnet_pos(x[1])), tagged))

    # lemmatize by the PoS
    lemmatized = list(map(lambda x: lemma.lemmatize(x[0], pos=x[1] if x[1] else wordnet.NOUN), wn_tagged))
    # lemma.lemmatize(wn_tagged[0], pos=wordnet.NOUN)

    return lemmatized

In [9]:
# lemmatize the data

X_lemmatized = list(map(lambda x: lemmatization(x), X))

In [10]:
X_lemmatized[0]

['get',
 'kq',
 'collection',
 'work',
 'window',
 'download',
 'patch',
 'http',
 'www',
 'sierrahelp',
 'com',
 'file',
 'patch',
 'kingsquest',
 'kqcollectiondbupdate',
 'exe',
 'set',
 'install',
 'c',
 'program',
 'file',
 'steam',
 'steamapps',
 'common',
 'king',
 'quest',
 'collection',
 'wherever',
 'steam',
 'save',
 'kq',
 'collection',
 'file',
 'program',
 'patch',
 'steam',
 'version',
 'kq',
 'collection',
 'run',
 'dos',
 'box',
 'confirm',
 'make',
 'game',
 'joy',
 'run',
 'win',
 'bit',
 'con',
 'run',
 'game',
 'dosbox',
 'shortcut',
 'patch',
 'create',
 'steam',
 'launcher',
 'work',
 'game',
 'mouse',
 'cursor',
 'become',
 'black',
 'white',
 'otherwise',
 'color']

In [11]:
# save the lematized data, as separate pickle file

import pickle

X_lemmatized_file = Path('02_Adventure_X_lemmatized.pkl')

with open(X_lemmatized_file, "wb") as f:
    pickle.dump(X_lemmatized, f)

In [9]:
# import pickle

# X_lemmatized_file = Path('dataset_cleaned_heartless_sampled_for_demo_X_lemmatized.pkl')

# with open(X_lemmatized_file, "rb") as f:
#     X_lemmatized = pickle.load(f)

# X_lemmatized[0]

['game',
 'bill',
 'scrabble',
 'meet',
 'sudoku',
 'unfortunately',
 'really',
 'multiple',
 'word',
 'scramble',
 'word',
 'certain',
 'set',
 'possible',
 'letter',
 'solution',
 'player',
 'must',
 'solve',
 'word',
 'cotemporaneously',
 'approach',
 'one',
 'might',
 'approach',
 'sudoku',
 'really',
 'possible',
 'play',
 'like',
 'electronic',
 'sudoku',
 'make',
 'notation',
 'possible',
 'answer',
 'board',
 'become',
 'nigh',
 'unplayable',
 'difficulty',
 'curve',
 'extremely',
 'uneven',
 'jump',
 'drop',
 'unexpectedly',
 'within',
 'difficulty',
 'head',
 'lexica',
 'also',
 'bill',
 'coffee',
 'break',
 'type',
 'game',
 'due',
 'finite',
 'number',
 'puzzle',
 'available',
 'although',
 'large',
 'number',
 'puzzle',
 'can',
 'not',
 'easily',
 'reset',
 'severely',
 'limits',
 'coffee',
 'break',
 'play',
 'random',
 'puzzle',
 'generator',
 'would',
 'improve',
 'limitation',
 'timer',
 'unnecessary',
 'feature',
 'can',
 'not',
 'reset',
 'puzzle',
 'replay',
 'can',

In [12]:
# use gensim to build a dictionary and train our LDAModel

id2word = gensim.corpora.Dictionary(X_lemmatized)

corpus = [id2word.doc2bow(text) for text in X_lemmatized]

In [13]:
N_TOPICS = 20

# Online LDA, how to effective train LDA models
# https://papers.nips.cc/paper_files/paper/2010/hash/71f6278d140af599e06ad9bf1ba03cb0-Abstract.html

lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                             id2word=id2word,
                                             num_topics=N_TOPICS,         # later can use grid search to find the best number of topics
                                             random_state=42,
                                             chunksize=2048,                # chunk size affects memory consumption, and updating speed (like DL batch_size). https://groups.google.com/g/gensim/c/FE7_FYSconA
                                             passes=10,                     # no. of passes over the whole corpus. If larger chunksize, then the passes should be larger too.
                                            #  alpha='auto',
                                             workers=3)     # workers = no. of cores (physical cores, but not logical threads)

visualize the data

In [14]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=10)
vis

/root/miniforge3/envs/fyp-test-wsl-tm/lib/python3.9/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.044602 -0.024239       1        1  19.986203
3      0.020951  0.108878       2        1  13.074423
7     -0.158658  0.092569       3        1   8.478790
19    -0.150205  0.212672       4        1   7.238119
12     0.145535  0.155642       5        1   7.221909
15     0.183181 -0.003043       6        1   6.133207
8      0.082811  0.286007       7        1   5.723680
0     -0.176151 -0.180624       8        1   5.382542
16    -0.286566 -0.018827       9        1   4.003921
10     0.310613  0.302857      10        1   3.116048
9      0.416957  0.022859      11        1   2.879765
11     0.063909  0.405742      12        1   2.863776
5     -0.221593 -0.399097      13        1   2.322992
4     -0.176480  0.432700      14        1   2.208396
2     -0.039827 -0.471050      15        1   2.128687
18    -0.479366  0.173659      16        1   1.895875
14     0.184992 -0.254707      17        1   1.754249
17    -0.433595 -0.220476      18        1   1.550168
13     0.294891 -0.444512      19        1   1.088460
1      0.463203 -0.177011      20        1   0.948792, topic_info=      Term          Freq         Total Category  logprob  loglift
18    game  1.188233e+06  1.188233e+06  Default  10.0000  10.0000
82    play  3.176020e+05  3.176020e+05  Default   9.0000   9.0000
19     get  2.456220e+05  2.456220e+05  Default   8.0000   8.0000
79     one  1.707120e+05  1.707120e+05  Default   7.0000   7.0000
161    fun  1.493270e+05  1.493270e+05  Default   6.0000   6.0000
...    ...           ...           ...      ...      ...      ...
1270   yes  5.060398e+03  1.556803e+04  Topic20  -3.6762   3.5340
1858   god  3.801293e+03  9.045645e+03  Topic20  -3.9623   3.7908
404     oh  4.082909e+03  1.202946e+04  Topic20  -3.8909   3.5772
5050   lie  3.001044e+03  5.438086e+03  Topic20  -4.1987   4.0633
18    game  3.122508e+03  1.188234e+06  Topic20  -4.1590  -1.2839

[472 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
10557      3  0.000199       ac
10557      4  0.999679       ac
607        1  0.346986      act
607        4  0.006163      act
607        6  0.646795      act
...      ...       ...      ...
432        1  0.079629    young
432       13  0.920218    young
1271      19  0.999859  youtube
4027       9  0.999907    zelda
2317      11  0.999872   zombie

[1454 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 4, 8, 20, 13, 16, 9, 1, 17, 11, 10, 12, 6, 5, 3, 19, 15, 18, 14, 2])

save model

we need to save the corpora.Dictionary and the LDA model

In [15]:
# save the LDA multicore model (and the corpora.Dictionary object) automatically

lda_save_folder = Path(f'lda_model_{datetime.now().strftime("%Y%m%d_%H%M%S")}')
if not lda_save_folder.exists():
    lda_save_folder.mkdir()

lda_model.save(str(lda_save_folder.joinpath('lda_model')))     # no need to add file extension

Evaluation

gensim provide functions to calculate, so we don't need to install octis (as the evaluation backend of octis also relies on gensim)

octis seems awesome for simple development, but it installs many packages ;(

In [16]:
# corpus = lemmatized words (?) (list of list of str)

# create a result object from the LDAMulticore model for octis evaluation
# referencing from https://github.com/MIND-Lab/OCTIS/blob/master/octis/models/LDA.py
# and guideline in README: https://github.com/MIND-Lab/OCTIS/tree/master
result_lda_online = {}
result_lda_online['topic-word-matrix'] = lda_model.get_topics()

top_words = 10
topics_output = []
for topic in result_lda_online["topic-word-matrix"]:
    top_k = np.argsort(topic)[-top_words:]
    top_k_words = list(reversed([id2word[i] for i in top_k]))
    topics_output.append(top_k_words)
result_lda_online["topics"] = topics_output

def _get_topic_document_matrix(lda_model, corpus, num_topics=10):
    """
    Return the topic representation of the
    corpus
    """

    id_corpus = corpus

    doc_topic_tuples = []
    for document in id_corpus:
        doc_topic_tuples.append(
            lda_model.get_document_topics(document, minimum_probability=0))

    topic_document = np.zeros((num_topics, len(doc_topic_tuples)))

    for ndoc in range(len(doc_topic_tuples)):
        document = doc_topic_tuples[ndoc]
        for topic_tuple in document:
            topic_document[topic_tuple[0]][ndoc] = topic_tuple[1]
    return topic_document

result_lda_online['topic-document-matrix'] = _get_topic_document_matrix(lda_model, corpus, num_topics=N_TOPICS)

In [17]:
lda_model.show_topics(num_topics=N_TOPICS, num_words=10, formatted=True, log=False)

[(0,
  '0.036*"level" + 0.016*"use" + 0.015*"weapon" + 0.014*"character" + 0.013*"enemy" + 0.013*"get" + 0.011*"skill" + 0.010*"quest" + 0.010*"item" + 0.010*"also"'),
 (1,
  '0.068*"cool" + 0.039*"valve" + 0.030*"cake" + 0.025*"yes" + 0.023*"f" + 0.020*"oh" + 0.020*"trine" + 0.019*"god" + 0.019*"potato" + 0.016*"game"'),
 (2,
  '0.044*"space" + 0.019*"ship" + 0.019*"pirate" + 0.016*"simulator" + 0.016*"planet" + 0.015*"around" + 0.015*"car" + 0.013*"robot" + 0.013*"fly" + 0.013*"moon"'),
 (3,
  '0.088*"game" + 0.027*"like" + 0.022*"play" + 0.019*"get" + 0.019*"say" + 0.017*"make" + 0.016*"go" + 0.014*"think" + 0.013*"know" + 0.013*"want"'),
 (4,
  '0.126*"worth" + 0.100*"buy" + 0.062*"price" + 0.060*"sale" + 0.053*"game" + 0.037*"money" + 0.029*"hour" + 0.024*"pay" + 0.024*"get" + 0.023*"full"'),
 (5,
  '0.034*"war" + 0.024*"life" + 0.018*"like" + 0.016*"dark" + 0.015*"lt" + 0.015*"cry" + 0.013*"call" + 0.013*"girl" + 0.013*"old" + 0.012*"movie"'),
 (6,
  '0.029*"game" + 0.011*"time" 

In [18]:
# setup: get the model's topics in their native ordering...
all_topics = lda_model.print_topics()
# ...then create a empty list per topic to collect the docs:
docs_per_topic = [[] for _ in all_topics]

# now, for every doc...
for doc_id, doc_bow in enumerate(corpus):
    # ...get its topics...
    doc_topics = lda_model.get_document_topics(doc_bow)
    # ...& for each of its topics...
    for topic_id, score in doc_topics:
        # ...add the doc_id & its score to the topic's doc list
        docs_per_topic[topic_id].append((doc_id, score))

In [19]:
# If you're interested in the top docs per topic, you can further sort each list's pairs by their score

for doc_list in docs_per_topic:
    doc_list.sort(key=lambda id_and_score: id_and_score[1], reverse=True)

In [20]:
print(docs_per_topic[0][:10])

[(550784, 0.9976069), (32135, 0.96833277), (464680, 0.9366626), (160998, 0.92081827), (566033, 0.90499055), (374601, 0.9049885), (457254, 0.8812413), (119969, 0.8812129), (38588, 0.8642835), (353436, 0.8642821)]


In [24]:
dataset.iloc[31170:31180]

,index,app_id,app_name,review_text,review_score,review_votes,genre_id,category_id
81877,87304,105600,Terraria,sick tired mother minecraft comparisons mother...,1,0,"[1, 25, 23, 3]","[2, 1, 49, 36, 9, 38, 22, 28, 29, 23, 41, 42, 43]"
81878,87305,105600,Terraria,best best best best best best best best best b...,1,0,"[1, 25, 23, 3]","[2, 1, 49, 36, 9, 38, 22, 28, 29, 23, 41, 42, 43]"
81879,87306,105600,Terraria,game makes want go fist cow lol,1,0,"[1, 25, 23, 3]","[2, 1, 49, 36, 9, 38, 22, 28, 29, 23, 41, 42, 43]"
81880,87307,105600,Terraria,game great recomend players like gaming advent...,1,0,"[1, 25, 23, 3]","[2, 1, 49, 36, 9, 38, 22, 28, 29, 23, 41, 42, 43]"
81881,87308,105600,Terraria,brilliant game one close heart one first games...,1,0,"[1, 25, 23, 3]","[2, 1, 49, 36, 9, 38, 22, 28, 29, 23, 41, 42, 43]"
81882,87309,105600,Terraria,awsome game updates great content fantastic be...,1,0,"[1, 25, 23, 3]","[2, 1, 49, 36, 9, 38, 22, 28, 29, 23, 41, 42, 43]"
81883,87310,105600,Terraria,terraria define game brought new vision terrar...,1,0,"[1, 25, 23, 3]","[2, 1, 49, 36, 9, 38, 22, 28, 29, 23, 41, 42, 43]"
81884,87311,105600,Terraria,game absolute dream many many things right yet...,1,0,"[1, 25, 23, 3]","[2, 1, 49, 36, 9, 38, 22, 28, 29, 23, 41, 42, 43]"
81885,87312,105600,Terraria,game fantastic nothing like minecraft many dif...,1,0,"[1, 25, 23, 3]","[2, 1, 49, 36, 9, 38, 22, 28, 29, 23, 41, 42, 43]"
81886,87313,105600,Terraria,game balls mcdonald play pit,1,0,"[1, 25, 23, 3]","[2, 1, 49, 36, 9, 38, 22, 28, 29, 23, 41, 42, 43]"


In [21]:
# show top 10 documents for each topic, also the name of the game
for topic_id, docs in enumerate(docs_per_topic):
    print(f'Topic {topic_id + 1}:')
    for doc_id, score in docs[:10]:
        print(f'Game: {dataset.iloc[doc_id]["app_name"]}')
        print(f'Doc ID: {doc_id}')
        print(f'Score: {score}')
        print(f'Doc: {dataset.iloc[doc_id]["review_text"]}')
        print()
    print('\n\n\n\n\n')

Topic 1:
Game: Devil May Cry 4
Doc ID: 550784
Score: 0.9976068735122681
Doc: sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick sword trick

In [ ]:
result_lda_online['topic-document-matrix'][0]

array([0.00047761, 0.01666767, 0.00135199, ..., 0.0027796 , 0.01250004,
       0.01000022])

In [ ]:
lda_model.get_topics().shape

(20, 410296)

In [ ]:
np.sum(result_lda_online['topic-document-matrix'], axis=0)

array([0.99999999, 1.00000001, 0.99999994, ..., 1.00000002, 1.00000001,
       1.00000008])

Evaluation

instead of using octis, we use gensim provided CoherenceModel object,  
as octis also uses this module for calculating the coherence scores

In [ ]:
from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(model=lda_model, texts=X_lemmatized, dictionary=id2word, coherence='c_v')
coherence_cv = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_cv)


Coherence Score:  0.4464862782742937


In [ ]:
# Compute Coherence Score using c_npmi
coherence_model_lda = CoherenceModel(model=lda_model, texts=X_lemmatized, dictionary=id2word, coherence='c_npmi')
coherence_npmi = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_npmi)


Coherence Score:  0.03243114218978375


INFERENCE

inference test

In [ ]:
# inference test

inference_test = ["well its been fun guys, but that's it, no more updates, that one was the last one, there is no longer going to be anymore content for this game anymore, there is no way to replay it as there won't be any updates, nope, that was it, the last update, nothing more, this game has no new ways to experience it as there is no more content updates, nothing new to freshen up the experience, its such a shame that this game has no replay-ability, once you beat the game there is like no point to playing again, as they said guys 1.2 will be they final update. nothing more after 1.2, there is no chance they will make another final update right? several years and final updates later: alright, thats it, no more updates we wont be getting anymore, thats it, nothing more, no more updates, for real this time... oh god, redigit made another tweet.",
                  "keeps forcing me to play it",
'''I will leave the cat here, so that everybody who passes by can pet it and give it a thumbs up and awards
　　　 　　／＞　　フ
　　　 　　| 　_　 _ l
　 　　 　／` ミ＿xノ
　　 　 /　　　 　 |
　　　 /　 ヽ　　 ﾉ
　 　 │　　|　|　|
　／￣|　　 |　|　|
　| (￣ヽ＿_ヽ_)__)
　＼二つ''']

inference_test = cleaning_strlist(inference_test)

inference_test = list(map(lambda x: lemmatization(x), inference_test))

corpus_test = [id2word.doc2bow(text) for text in inference_test]

test_output = lda_model[corpus_test]

test_output

In [ ]:
inference_test[-1]

['leave',
 'cat',
 'everybody',
 'pass',
 'pet',
 'give',
 'thumb',
 'award',
 'l',
 'x']

In [ ]:
# test inference

corpus_test = [id2word.doc2bow(text) for text in inference_test]

output_test = lda_model[corpus_test]

for i in range(len(output_test)):
    # print(sorted(test_output[i], key=lambda x: x[1], reverse=True))
    print(sorted(output_test[i], key=lambda x: x[1], reverse=True))

[(3, 0.60644966), (11, 0.15789744), (13, 0.123749346), (10, 0.08829053), (8, 0.013170155)]
[(10, 0.5038039), (6, 0.27117985), (0, 0.012500903), (1, 0.012500903), (2, 0.012500903), (3, 0.012500903), (4, 0.012500903), (5, 0.012500903), (7, 0.012500903), (8, 0.012500903), (9, 0.012500903), (11, 0.012500903), (12, 0.012500903), (13, 0.012500903), (14, 0.012500903), (15, 0.012500903), (16, 0.012500903), (17, 0.012500903), (18, 0.012500903), (19, 0.012500903)]
[(16, 0.50347227), (11, 0.21316677), (1, 0.11489906), (18, 0.095704)]


load model (both corpora Dictionary and the LDA model)

In [ ]:
del id2word
del lda_model

model_datetime = datetime(2024, 1, 15, 0, 21, 57)
lda_save_folder = Path(f'lda_model_{model_datetime.strftime("%Y%m%d_%H%M%S")}')

# id2word_load = gensim.corpora.Dictionary.load('lda_model.id2word')
id2word_l = gensim.corpora.Dictionary.load(str(lda_save_folder.joinpath('lda_model.id2word')))

lda_model_l = gensim.models.ldamulticore.LdaMulticore.load(str(lda_save_folder.joinpath('lda_model')))

In [ ]:
corpus_test2 = [id2word_l.doc2bow(text) for text in inference_test]

output_test2 = lda_model_l[corpus_test2]

for i in range(len(output_test2)):
    print(sorted(output_test2[i], key=lambda x: x[1], reverse=True))

[(1, 0.32744843), (16, 0.2903514), (15, 0.123242974), (6, 0.09021147), (18, 0.09014924), (5, 0.030303197), (19, 0.023578195), (7, 0.016134478)]
[(1, 0.49804363), (10, 0.2769175), (0, 0.012502159), (2, 0.012502159), (3, 0.012502159), (4, 0.012502159), (5, 0.012502159), (6, 0.012502159), (7, 0.012502159), (8, 0.012502159), (9, 0.012502159), (11, 0.012502159), (12, 0.012502159), (13, 0.012502159), (14, 0.012502159), (15, 0.012502159), (16, 0.012502159), (17, 0.012502159), (18, 0.012502159), (19, 0.012502159)]
[(13, 0.27624068), (7, 0.20502365), (14, 0.13374868), (17, 0.105004296), (11, 0.104988545), (2, 0.10497881)]
